----------Librerias utilizadas----------    

In [2]:
pip install qvd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.9/578.9 KB 22.6 MB/s eta 0:00:00


In [72]:
import re
import pandas as pd
from qvd import qvd_reader
import numpy as np

----------lectura y filtro del excel. Pasa al dfDatos----------

In [178]:
df_param = pd.read_excel("Parametros.xlsx",sheet_name="Validaciones")
df_param_ = df_param.loc[df_param['Tipo Validacion'] == 'Formato']
tablas = df_param_.drop_duplicates(subset=['Esquema', 'Tabla'])
name_tablas = tablas.reset_index()
name_tabla = name_tablas.iloc[0]["Tabla"]
tablas = np.array(tablas["Esquema"]+"_"+tablas["Tabla"])
tablas_campo = df_param_["Esquema"]+"_"+df_param_["Tabla"]+"."+df_param_["Campo"]
"""df_param_["Valores"]=df_param_["Valores"].replace("TipoDato", "hasDataType")
df_param_["Valores"]=df_param_["Valores"].replace("MinLength", "hasMinLength")
df_param_["Valores"]=df_param_["Valores"].replace("MaxLength", "hasMaxLength")
df_param_["Valores"]=df_param_["Valores"].replace("Email", "containsEmail")
df_param_["Valores"]=df_param_["Valores"].replace("MaxNumRegistros", "hasSize")
df_param_["Valores"]=df_param_["Valores"].replace("CreditCardNumber", "containsCreditCardNumber")
df_param_["Valores"]=df_param_["Valores"].replace("URL", "containsURL")
df_param_["Valores"]=df_param_["Valores"].replace("SocialSecurityNumber", "containsSocialSecurityNumber")"""
df_param_ = df_param_.loc[df_param_['Tabla'] == name_tabla]





In [179]:
df_param_ 


,Esquema,Tabla,Campo,Tipo Validacion,Valores,Tipo_dato,Formato,Prioridad,Dimension,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
3,dbo,Prueba,Cargo,Formato,Fecha,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,dbo,Prueba,Email,Formato,Regex,NaN,^([a-zA-Z0-9_\.-]+)([\da-zA-Z\.-]+)\.([a-zA-Z\...,NaN,NaN,NaN,NaN,NaN,NaN
5,dbo,Prueba,celular,Formato,Numero,EnteroPositivo,3,NaN,NaN,NaN,NaN,NaN,NaN
6,dbo,Prueba,fecha_Cumpleaños,Formato,Fecha,aa-mm-dd,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,dbo,Prueba,sitio_web,Formato,URL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [180]:
dfqvd = pd.read_excel(tablas[0]+'.xlsx')


In [181]:
dfqvd

,Colaborador,Cargo,Email,celular,fecha_Cumpleaños,sitio_web
0,Alex pierre Arce Sánchez,Consultor de Datos,gmxaqqqhotmail.com,-444444.785,11/12/1988,cswcca
1,Guadalupe martinez malacios,Analista Funcional,gmartinez@hotmail.com,444445,1993-10-08,as as
2,hermes Vizconde Olivares,Scrum Master Specialist,hvizcondeo88@outlook.com.pe,444446,23-13-20,112232443
3,Kevin Tacco,Data Engineer Specialist,kevin.tacco@rimac.com.co,"444447,333",2992-04-11,#####.com
4,Milagros Madeleine Minano Ochoa,Analista Aseguramiento de Calidad,milagros.minano@gmail.com,444448,91-12-32,https://www.analyticslane.com/2022/02/21/panda...
5,Noemi Obregón Vargas,Analista Funcional,88nyobregon@indracompany.com,444449,17 de enero de 1990,www.google.com
6,Rommy Elizabeth Ortiz Tupac Yupanqui,Jefe de Data,"/*%!#""@vooxell.com",444450,88-10-08,eeee.dsfddsfs.com.co
7,Luis Angel Castillo Dominguez,Ingeniero de Datos,luisangel_castillod@gmail.com,adfcqfcqvc,05-05-1985,https://www.youtube.com/watch?v=XLuA5HZ8h6k&ab...
8,NaN,Ingeniero de Datos,956384@zoluxiones.com,444452,enero/29/1986,https://www.google.com/search?q=expresion+regu...
9,Wilson Andrey Cárdenas Henao,Analista Aseguramiento de Calidad,wcardenash@choucairtesting.com,444453,8 de enero de 1987,https://cwcvwcwwcw


----------Creacion de rutas que se usan en el codigo----------

-------- Ciclo for recorre el df datos en busca de las tablas a validar y lectura de qvd---------

----------Creacion de variables vacias que se llenaran con las validaciones de expresiones regulares----------

In [182]:
listas1 =[]
for i in range(len(df_param_)):
  if df_param_.iloc[i]["Valores"] == 'Numero':
    if df_param_.iloc[i]["Tipo_dato"] == 'Entero':
      campo = (df_param_.iloc[i]["Campo"])
      tabla = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"])
      dfqvd[tabla+"."+campo] = dfqvd[campo]
      lista = (dfqvd[campo])
      lista = lista.to_numpy().tolist()
      for j in range(len(lista)):
        a =(lista[j])
        if re.fullmatch(r'^[\-]?\d*$', str(a)):
          dfqvd[tabla+"."+campo][j] = 'valido'
        else:
          dfqvd[tabla+"."+campo][j] = 'no valido'
#-----------------------
    elif df_param_.iloc[i]["Tipo_dato"] == 'Decimal':
      num_decimales = df_param_.iloc[i]["Formato"]
      b = '^-?\d*([\.\,]\d{'+str(num_decimales)+'})$'
      print (b)
      campo = (df_param_.iloc[i]["Campo"])
      tabla = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"])
      dfqvd[tabla+"."+campo] = dfqvd[campo]
      lista = (dfqvd[campo])
      lista = lista.to_numpy().tolist()
      for j in range(len(lista)):
        a =(lista[j])
        if re.fullmatch(b, str(a)):
          dfqvd[tabla+"."+campo][j] = 'valido'
        else:
          dfqvd[tabla+"."+campo][j] = 'no valido' 
        
   #----------------------- 
    elif df_param_.iloc[i]["Tipo_dato"] == 'EnteroPositivo':
      campo = (df_param_.iloc[i]["Campo"])
      tabla = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"])
      dfqvd[tabla+"."+campo] = dfqvd[campo]
      lista = (dfqvd[campo])
      lista = lista.to_numpy().tolist()
      for j in range(len(lista)):
        a =(lista[j])
        if re.fullmatch(r'^\d*([\.\,]\d+)?$', str(a)):
          dfqvd[tabla+"."+campo][j] = 'valido'
        else:
          dfqvd[tabla+"."+campo][j] = 'no valido'
#-----------------------
  if df_param_.iloc[i]["Valores"] == 'Email':
    campo = (df_param_.iloc[i]["Campo"])
    tabla = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"])
    dfqvd[tabla+"."+campo] = dfqvd[campo]
    lista = (dfqvd[campo])
    lista = lista.to_numpy().tolist()
    for j in range(len(lista)):
      a =(lista[j])
      b = '^[A-Za-z0-9][\w\.-]{1,50}\+?[^\.\_\-]@[A-z0-9]+(\.\w{2,3})?(\.\w{2,3})?(\.\w{2,3})?$'
      
      if re.fullmatch(b, str(a)):
          dfqvd[tabla+"."+campo][j] = 'valido'
      else:
        dfqvd[tabla+"."+campo][j] = 'no valido'
  #-----------------------
  if df_param_.iloc[i]["Valores"] == 'URL':
    campo = (df_param_.iloc[i]["Campo"])
    tabla = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"])
    dfqvd[tabla+"."+campo] = dfqvd[campo]
    lista = (dfqvd[campo])
    lista = lista.to_numpy().tolist()
    for j in range(len(lista)):
      a =(lista[j])
      b = '^(https?:\/\/)?([\da-zA-Z\.-]+)\.([a-zA-Z\.]{2,6})([\/\w \.-]*[^n])*\/?$'
      if re.fullmatch(b, str(a)):
          dfqvd[tabla+"."+campo][j] = 'valido'
      else:
        dfqvd[tabla+"."+campo][j] = 'no valido'
#-----------------------
  if df_param_.iloc[i]["Valores"] == 'Fecha':
   if df_param_.iloc[i]["Tipo_dato"] == 'dd/mm/aaaa':
      campo = (df_param_.iloc[i]["Campo"])
      tabla = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"])
      dfqvd[tabla+"."+campo] = dfqvd[campo]
      lista = (dfqvd[campo])
      lista = lista.to_numpy().tolist()
      for j in range(len(lista)):
        a =lista[j]
        sacar_año = a.split('/')
        print(sacar_año)
        b = '([0-2][0-9]|3[0-1])(\/)(0[1-9]|1[0-2])(\/)([1]{1}[9]{1}[0-9]{2})'
        c = '([0-2][0-9]|3[0-1])(\/)(0[1-9]|1[0-2])(\/)([2]{1}[0]{1}[0-9]{2})'
        
        
        if len(sacar_año)>1:
          
          if int(sacar_año[2]) < 2000 :
            if re.fullmatch(b, str(a)):
              dfqvd[tabla+"."+campo][j] = 'valido'
            else:
              dfqvd[tabla+"."+campo][j] = 'no valido '

          if int(sacar_año[2]) >= 2000:
            if re.fullmatch(c, a):
              dfqvd[tabla+"."+campo][j] = 'valido'
            else:
              dfqvd[tabla+"."+campo][j] = 'no valido'
        else:
          dfqvd[tabla+"."+campo][j] = 'no valido'


  #-----------------------
  if df_param_.iloc[i]["Valores"] == 'Fecha':
   if df_param_.iloc[i]["Tipo_dato"] == 'dd-mm-aaaa':
      campo = (df_param_.iloc[i]["Campo"])
      tabla = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"])
      dfqvd[tabla+"."+campo] = dfqvd[campo]
      lista = (dfqvd[campo])
      lista = lista.to_numpy().tolist()
      for j in range(len(lista)):
        a =lista[j]
        sacar_año = a.split('-')
        print(sacar_año)
        b = '([0-2][0-9]|3[0-1])(\-)(0[1-9]|1[0-2])(\-)([1]{1}[9]{1}[0-9]{2})'
        c = '([0-2][0-9]|3[0-1])(\-)(0[1-9]|1[0-2])(\-)([2]{1}[0]{1}[0-9]{2})'
        
        
        if len(sacar_año)>1:
          
          if int(sacar_año[2]) < 2000 :
            if re.fullmatch(b, str(a)):
              dfqvd[tabla+"."+campo][j] = 'valido'
            else:
              dfqvd[tabla+"."+campo][j] = 'no valido '

          if int(sacar_año[2]) >= 2000:
            if re.fullmatch(c, a):
              dfqvd[tabla+"."+campo][j] = 'valido'
            else:
              dfqvd[tabla+"."+campo][j] = 'no valido'
        else:
          dfqvd[tabla+"."+campo][j] = 'no valido'

#-----------------------
  if df_param_.iloc[i]["Valores"] == 'Fecha':
   if df_param_.iloc[i]["Tipo_dato"] == 'mm/dd/aaaa':
      campo = (df_param_.iloc[i]["Campo"])
      tabla = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"])
      dfqvd[tabla+"."+campo] = dfqvd[campo]
      lista = (dfqvd[campo])
      lista = lista.to_numpy().tolist()
      for j in range(len(lista)):
        a =lista[j]
        sacar_año = a.split('/')
        
        b = '(0[1-9]|1[0-2])(\/)([0-2][0-9]|3[0-1])(\/)([1]{1}[9]{1}[0-9]{2})'
        c = '(0[1-9]|1[0-2])(\/)([0-2][0-9]|3[0-1])(\/)([2]{1}[0]{1}[0-9]{2})'
        
        
        if len(sacar_año)>1:
          
          if int(sacar_año[2]) < 2000 :
            if re.fullmatch(b, str(a)):
              dfqvd[tabla+"."+campo][j] = 'valido'
            else:
              dfqvd[tabla+"."+campo][j] = 'no valido '

          if int(sacar_año[2]) >= 2000:
            if re.fullmatch(c, a):
              dfqvd[tabla+"."+campo][j] = 'valido'
            else:
              dfqvd[tabla+"."+campo][j] = 'no valido'
        else:
          dfqvd[tabla+"."+campo][j] = 'no valido'

#----------------------------------------------

  if df_param_.iloc[i]["Valores"] == 'Fecha':
   if df_param_.iloc[i]["Tipo_dato"] == 'mm-dd-aaaa':
      campo = (df_param_.iloc[i]["Campo"])
      tabla = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"])
      dfqvd[tabla+"."+campo] = dfqvd[campo]
      lista = (dfqvd[campo])
      lista = lista.to_numpy().tolist()
      for j in range(len(lista)):
        a =lista[j]
        sacar_año = a.split('-')
        
        b = '(0[1-9]|1[0-2])(\-)([0-2][0-9]|3[0-1])(\-)([1]{1}[9]{1}[0-9]{2})'
        c = '(0[1-9]|1[0-2])(\-)([0-2][0-9]|3[0-1])(\-)([2]{1}[0]{1}[0-9]{2})'
        
        
        if len(sacar_año)>1:
          
          if int(sacar_año[2]) < 2000 :
            if re.fullmatch(b, str(a)):
              dfqvd[tabla+"."+campo][j] = 'valido'
            else:
              dfqvd[tabla+"."+campo][j] = 'no valido '

          if int(sacar_año[2]) >= 2000:
            if re.fullmatch(c, a):
              dfqvd[tabla+"."+campo][j] = 'valido'
            else:
              dfqvd[tabla+"."+campo][j] = 'no valido'
        else:
          dfqvd[tabla+"."+campo][j] = 'no valido'

#-----------------------
  if df_param_.iloc[i]["Valores"] == 'Fecha':
   if df_param_.iloc[i]["Tipo_dato"] == 'aaaa/mm/dd':
      campo = (df_param_.iloc[i]["Campo"])
      tabla = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"])
      dfqvd[tabla+"."+campo] = dfqvd[campo]
      lista = (dfqvd[campo])
      lista = lista.to_numpy().tolist()
      for j in range(len(lista)):
        a =lista[j]
        sacar_año = a.split('/')
        b = '([1]{1}[9]{1}[0-9]{2})(\/)(0[1-9]|1[0-2])(\/)([0-2][0-9]|3[0-1])'
        c = '([2]{1}[0]{1}[0-9]{2})(\/)(0[1-9]|1[0-2])(\/)([0-2][0-9]|3[0-1])'
        
        
        if len(sacar_año)>1:
          

          if re.fullmatch('\d+', sacar_año[0]):
            
            if int(sacar_año[0]) < 2000 :
              if re.fullmatch(b, str(a)):
                dfqvd[tabla+"."+campo][j] = 'valido'
              else:
                dfqvd[tabla+"."+campo][j] = 'no valido'

            if int(sacar_año[0]) >= 2000:
              

              if re.fullmatch(c, a):
                dfqvd[tabla+"."+campo][j] = 'valido'
              else:
                dfqvd[tabla+"."+campo][j] = 'no valido'
          else:
            dfqvd[tabla+"."+campo][j] = 'no valido'
        else:
          dfqvd[tabla+"."+campo][j] = 'no valido'


#-----------------------
  if df_param_.iloc[i]["Valores"] == 'Fecha':
   if df_param_.iloc[i]["Tipo_dato"] == 'aaaa-mm-dd':
    campo = (df_param_.iloc[i]["Campo"])
    tabla = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"])
    dfqvd[tabla+"."+campo] = dfqvd[campo]
    lista = (dfqvd[campo])
    lista = lista.to_numpy().tolist()
    for j in range(len(lista)):
      a =lista[j]
      sacar_año = a.split('-')
      b = '([1]{1}[9]{1}[0-9]{2})(\-)(0[1-9]|1[0-2])(\-)([0-2][0-9]|3[0-1])'
      c = '([2]{1}[0]{1}[0-9]{2})(\-)(0[1-9]|1[0-2])(\-)([0-2][0-9]|3[0-1])'
      
      
      if len(sacar_año)>1:
        

        if re.fullmatch('\d+', sacar_año[0]):
          
          if int(sacar_año[0]) < 2000 :
            if re.fullmatch(b, str(a)):
              dfqvd[tabla+"."+campo][j] = 'valido'
            else:
              dfqvd[tabla+"."+campo][j] = 'no valido'

          if int(sacar_año[0]) >= 2000:
            

            if re.fullmatch(c, a):
              dfqvd[tabla+"."+campo][j] = 'valido'
            else:
              dfqvd[tabla+"."+campo][j] = 'no valido'
        else:
          dfqvd[tabla+"."+campo][j] = 'no valido'
      else:
        dfqvd[tabla+"."+campo][j] = 'no valido'
#------------------------------------------

  if df_param_.iloc[i]["Valores"] == 'Fecha':
   if df_param_.iloc[i]["Tipo_dato"] == 'aa/mm/dd':
    campo = (df_param_.iloc[i]["Campo"])
    tabla = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"])
    dfqvd[tabla+"."+campo] = dfqvd[campo]
    lista = (dfqvd[campo])
    lista = lista.to_numpy().tolist()
    for j in range(len(lista)):
      a =lista[j]
      b = '([0-9]{2})(\/)(0[1-9]|1[0-2])(\/)([0-2][0-9]|3[0-1])'      
      
      if re.fullmatch(b,a):
        dfqvd[tabla+"."+campo][j] = 'valido'
      else:
        dfqvd[tabla+"."+campo][j] = 'no valido'

#---------------------------------

  if df_param_.iloc[i]["Valores"] == 'Fecha':
   if df_param_.iloc[i]["Tipo_dato"] == 'aa-mm-dd':
    campo = (df_param_.iloc[i]["Campo"])
    tabla = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"])
    dfqvd[tabla+"."+campo] = dfqvd[campo]
    lista = (dfqvd[campo])
    lista = lista.to_numpy().tolist()
    for j in range(len(lista)):
      a =lista[j]
      b = '([0-9]{2})(\-)(0[1-9]|1[0-2])(\-)([0-2][0-9]|3[0-1])'      
      
      if re.fullmatch(b,a):
        dfqvd[tabla+"."+campo][j] = 'valido'
      else:
        dfqvd[tabla+"."+campo][j] = 'no valido'


#--------------------------------
  if df_param_.iloc[i]["Valores"] == 'Regex':
    campo = (df_param_.iloc[i]["Campo"])
    tabla = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"])
    dfqvd[tabla+"."+campo] = dfqvd[campo]
    lista = (dfqvd[campo])
    lista = lista.to_numpy().tolist()
    for j in range(len(lista)):
      a =(lista[j])
      b = df_param_.iloc[i]["Formato"]
      if re.fullmatch(str(b), str(a)):
          dfqvd[tabla+"."+campo][j] = 'valido'
      else:
        dfqvd[tabla+"."+campo][j] = 'no valido'

dfqvd


,Colaborador,Cargo,Email,celular,fecha_Cumpleaños,sitio_web,dbo_Prueba.Email,dbo_Prueba.celular,dbo_Prueba.fecha_Cumpleaños,dbo_Prueba.sitio_web
0,Alex pierre Arce Sánchez,Consultor de Datos,gmxaqqqhotmail.com,-444444.785,11/12/1988,cswcca,valido,no valido,no valido,no valido
1,Guadalupe martinez malacios,Analista Funcional,gmartinez@hotmail.com,444445,1993-10-08,as as,no valido,valido,no valido,no valido
2,hermes Vizconde Olivares,Scrum Master Specialist,hvizcondeo88@outlook.com.pe,444446,23-13-20,112232443,no valido,valido,no valido,no valido
3,Kevin Tacco,Data Engineer Specialist,kevin.tacco@rimac.com.co,"444447,333",2992-04-11,#####.com,no valido,valido,no valido,no valido
4,Milagros Madeleine Minano Ochoa,Analista Aseguramiento de Calidad,milagros.minano@gmail.com,444448,91-12-32,https://www.analyticslane.com/2022/02/21/panda...,no valido,valido,no valido,valido
5,Noemi Obregón Vargas,Analista Funcional,88nyobregon@indracompany.com,444449,17 de enero de 1990,www.google.com,no valido,valido,no valido,valido
6,Rommy Elizabeth Ortiz Tupac Yupanqui,Jefe de Data,"/*%!#""@vooxell.com",444450,88-10-08,eeee.dsfddsfs.com.co,no valido,valido,valido,valido
7,Luis Angel Castillo Dominguez,Ingeniero de Datos,luisangel_castillod@gmail.com,adfcqfcqvc,05-05-1985,https://www.youtube.com/watch?v=XLuA5HZ8h6k&ab...,no valido,no valido,no valido,valido
8,NaN,Ingeniero de Datos,956384@zoluxiones.com,444452,enero/29/1986,https://www.google.com/search?q=expresion+regu...,no valido,valido,no valido,valido
9,Wilson Andrey Cárdenas Henao,Analista Aseguramiento de Calidad,wcardenash@choucairtesting.com,444453,8 de enero de 1987,https://cwcvwcwwcw,no valido,valido,no valido,no valido


In [156]:
a = '2095/02/25'
b = '([2]{1}[0]{1}[0-9]{2})(\/)(0[1-9]|1[0-2])(\/)([0-2][0-9]|3[0-1])'
if re.fullmatch(b, a):
     print('valido')
else:
    print('no valido')

valido


In [187]:
dfqvd.to_excel('validacion_'+tablas[0]+'.xlsx', sheet_name='example')

In [177]:
a ='17 de enero de 1990	'

b = '([0-2][0-9]|3[0-1])(\/|-)(0[1-9]|1[0-2])(\/|-)(\d{4})'

if re.fullmatch(str(b),str(a)):
  print('valido')
else:
  print('no valido')



no valido


In [25]:
dfqvd

,Colaborador,Cargo,Email,celular,fecha_Cumpleaños,ValoresValidacion,Campo
0,Alex pierre Arce Sánchez,Consultor de Datos,gmartinez@hotmail.com,444444,11/12/1988,,
1,Guadalupe martinez malacios,Analista Funcional,gmartinez@hotmail.com,444445,1993/25/08,,
2,hermes Vizconde Olivares,Scrum Master Specialist,hvizcondeo88@outlook.com.pe,444446,15-sep-90,,
3,Kevin Tacco,Data Engineer Specialist,kevin.tacco@rimac.com.co,444447,1992-04-11,,
4,Milagros Madeleine Minano Ochoa,Analista Aseguramiento de Calidad,milagros.minano@gmail.com,444448,10-12-91,,
5,Noemi Obregón Vargas,Analista Funcional,88nyobregon@indracompany.com,444449,17 de enero de 1990,,
6,Rommy Elizabeth Ortiz Tupac Yupanqui,Jefe de Data,93rortizt65@rimac.com.pe,444450,10/10/88,,
7,Luis Angel Castillo Dominguez,Ingeniero de Datos,luisangel_castillod@gmail.com,444451,18/05/1985,,
8,NaN,Ingeniero de Datos,956384@zoluxiones.com,444452,enero/29/1986,,
9,Wilson Andrey Cárdenas Henao,Analista Aseguramiento de Calidad,wcardenash@choucairtesting.com,444453,8 de enero de 1987,,


----------Validación de expresión regular----------

In [180]:

# Ciclo recorre el dfDatos y asigna a las variables campo, Formato y tabla el contenido de dfDatos para la validación
listas1 =[]
for i in range(len(df_param_)):
    if df_param_.iloc[i]["Valores"] == "Email":
      campo = (df_param_.iloc[i]["Campo"])
      Formato = '^([a-zA-Z0-9_\.-]+)@([\da-zA-Z\.-]+)\.([a-zA-Z\.]{2,20}[^n])$'
      tabla = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"])
      print(tabla, campo)
      print(tabla+"."+campo)
      dfqvd[tabla+"."+campo] = dfqvd[campo]
      dfqvd = dfqvd.replace({campo: Formato},{campo:"Valido"}, regex=True)
      tabla_campo = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"]+"."+df_param_.iloc[i]["Campo"])
      #Lleamos los valores nulos con "No valido"
      dfqvd1 = dfqvd.fillna("Nulo")
      listas1.append([tabla_campo,campo])

    elif df_param_.iloc[i]["Valores"] == "URL":
      campo = (df_param_.iloc[i]["Campo"])
      Formato = '^(https?:\/\/)?([\da-zA-Z\.-]+)\.([a-zA-Z\.]{2,6})([\/\w \.-]*[^n])*\/?$'
      tabla = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"])
      print(tabla, campo)
      print(tabla+"."+campo)
      dfqvd[tabla+"."+campo] = dfqvd[campo]
      dfqvd = dfqvd.replace({campo: Formato},{campo:"Valido"}, regex=True)
      tabla_campo = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"]+"."+df_param_.iloc[i]["Campo"])
      #Lleamos los valores nulos con "No valido"
      dfqvd1 = dfqvd.fillna("Nulo")
      listas1.append([tabla_campo,campo])

    elif df_param_.iloc[i]["Valores"] == "TarjetaCredito":
      campo = (df_param_.iloc[i]["Campo"])
      Formato = '^[973][0-9]{15}|[973][0-9]{3}-[0-9]{4}-[0-9]{4}-[0-9]{4}$'
      tabla = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"])
      print(tabla, campo)
      print(tabla+"."+campo)
      dfqvd[tabla+"."+campo] = dfqvd[campo]
      dfqvd = dfqvd.replace({campo: Formato},{campo:"Valido"}, regex=True)
      tabla_campo = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"]+"."+df_param_.iloc[i]["Campo"])
      #Lleamos los valores nulos con "No valido"
      dfqvd1 = dfqvd.fillna("Nulo")
      listas1.append([tabla_campo,campo])

    elif df_param_.iloc[i]["Valores"] == "TipoDato":
      if df_param_.iloc[i]["Tipo_dato"] == 'Texto':
        campo = (df_param_.iloc[i]["Campo"])
        Formato = '([A-Za-z0-9_]|[^A-Za-z0-9_])'
        tabla = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"])
        print(tabla, campo)
        print(tabla+"."+campo)
        dfqvd[tabla+"."+campo] = dfqvd[campo]
        #dfqvd = dfqvd.replace({campo: Formato},{campo:"Valido"}, regex=True)
        dfqvd = dfqvd.replace({campo: Formato},{campo:"Valido"}, regex=True)
        tabla_campo = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"]+"."+df_param_.iloc[i]["Campo"])
        #Lleamos los valores nulos con "No valido"
        dfqvd1 = dfqvd.fillna("Nulo")
        listas1.append([tabla_campo,campo])
      elif df_param_.iloc[i]["Tipo_dato"] == 'Numerico':
        campo = (df_param_.iloc[i]["Campo"])
        Formato = '[0-9]{3,30}'
        tabla = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"])
        print(tabla, campo)
        print(tabla+"."+campo)
        dfqvd[tabla+"."+campo] = dfqvd[campo]
        #dfqvd = dfqvd.replace({campo: Formato},{campo:"Valido"}, regex=True)
        dfqvd = dfqvd.replace({campo: Formato},{campo:"Valido"}, regex=True)
        tabla_campo = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"]+"."+df_param_.iloc[i]["Campo"])
        #Lleamos los valores nulos con "No valido"
        dfqvd1 = dfqvd.fillna("Nulo")
        listas1.append([tabla_campo,campo])
      elif df_param_.iloc[i]["Tipo_dato"] == 'Decimal':
        campo = (df_param_.iloc[i]["Campo"])
        Formato = '[+-]?\d*\.?\d+'
        tabla = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"])
        print(tabla, campo)
        print(tabla+"."+campo)
        dfqvd[tabla+"."+campo] = dfqvd[campo]
        #dfqvd = dfqvd.replace({campo: Formato},{campo:"Valido"}, regex=True)
        dfqvd = dfqvd.replace({campo: Formato},{campo:"Valido"}, regex=True)
        tabla_campo = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"]+"."+df_param_.iloc[i]["Campo"])
        #Lleamos los valores nulos con "No valido"
        dfqvd1 = dfqvd.fillna("Nulo")
        listas1.append([tabla_campo,campo])

    elif df_param_.iloc[i]["Valores"] == "Fecha":
      if df_param_.iloc[i]["Tipo_dato"] == 'dd/mm/aaaa':
        campo = (df_param_.iloc[i]["Campo"])
        Formato = '^([0-2][0-9]|3[0-1])(\/|-)(0[1-9]|1[0-2])\2(\d{4})$'
        tabla = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"])
        print(tabla, campo)
        print(tabla+"."+campo)
        dfqvd[tabla+"."+campo] = dfqvd[campo]
        #dfqvd = dfqvd.replace({campo: Formato},{campo:"Valido"}, regex=True)
        dfqvd = dfqvd.replace({campo: Formato},{campo:"Valido"}, regex=True)
        tabla_campo = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"]+"."+df_param_.iloc[i]["Campo"])
        #Lleamos los valores nulos con "No valido"
        dfqvd1 = dfqvd.fillna("Nulo")
        listas1.append([tabla_campo,campo])
      elif df_param_.iloc[i]["Tipo_dato"] == 'dd-mm-aaaa':
        campo = (df_param_.iloc[i]["Campo"])
        Formato = '^([0-2][0-9]|3[0-1])(\/|-)(0[1-9]|1[0-2])\2(\d{4})$'
        tabla = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"])
        print(tabla, campo)
        print(tabla+"."+campo)
        dfqvd[tabla+"."+campo] = dfqvd[campo]
        #dfqvd = dfqvd.replace({campo: Formato},{campo:"Valido"}, regex=True)
        dfqvd = dfqvd.replace({campo: Formato},{campo:"Valido"}, regex=True)
        tabla_campo = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"]+"."+df_param_.iloc[i]["Campo"])
        #Lleamos los valores nulos con "No valido"
        dfqvd1 = dfqvd.fillna("Nulo")
        listas1.append([tabla_campo,campo])
      elif df_param_.iloc[i]["Tipo_dato"] == 'mm/dd/aaaa':
        campo = (df_param_.iloc[i]["Campo"])
        Formato = '^(0[1-9]|1[0-2])(\/|-)([0-2][0-9]|3[0-1])\2(\d{4})$'
        tabla = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"])
        print(tabla, campo)
        print(tabla+"."+campo)
        dfqvd[tabla+"."+campo] = dfqvd[campo]
        #dfqvd = dfqvd.replace({campo: Formato},{campo:"Valido"}, regex=True)
        dfqvd = dfqvd.replace({campo: Formato},{campo:"Valido"}, regex=True)
        tabla_campo = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"]+"."+df_param_.iloc[i]["Campo"])
        #Lleamos los valores nulos con "No valido"
        dfqvd1 = dfqvd.fillna("Nulo")
        listas1.append([tabla_campo,campo])
      elif df_param_.iloc[i]["Tipo_dato"] == 'mm-dd-aaaa':
        campo = (df_param_.iloc[i]["Campo"])
        Formato = '^(0[1-9]|1[0-2])(\/|-)([0-2][0-9]|3[0-1])\2(\d{4})$'
        tabla = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"])
        print(tabla, campo)
        print(tabla+"."+campo)
        dfqvd[tabla+"."+campo] = dfqvd[campo]
        #dfqvd = dfqvd.replace({campo: Formato},{campo:"Valido"}, regex=True)
        dfqvd = dfqvd.replace({campo: Formato},{campo:"Valido"}, regex=True)
        tabla_campo = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"]+"."+df_param_.iloc[i]["Campo"])
        #Lleamos los valores nulos con "No valido"
        dfqvd1 = dfqvd.fillna("Nulo")
        listas1.append([tabla_campo,campo])
      elif df_param_.iloc[i]["Tipo_dato"] == 'aaaa/mm/dd':
        campo = (df_param_.iloc[i]["Campo"])
        Formato = '^(\d{4})(\/|-)(0[1-9]|1[0-2])\2([0-2][0-9]|3[0-1])$'
        tabla = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"])
        print(tabla, campo)
        print(tabla+"."+campo)
        dfqvd[tabla+"."+campo] = dfqvd[campo]
        #dfqvd = dfqvd.replace({campo: Formato},{campo:"Valido"}, regex=True)
        dfqvd = dfqvd.replace({campo: Formato},{campo:"Valido"}, regex=True)
        tabla_campo = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"]+"."+df_param_.iloc[i]["Campo"])
        #Lleamos los valores nulos con "No valido"
        dfqvd1 = dfqvd.fillna("Nulo")
        listas1.append([tabla_campo,campo])
      elif df_param_.iloc[i]["Tipo_dato"] == 'aaaa-mm-dd':
        campo = (df_param_.iloc[i]["Campo"])
        Formato = '^(\d{4})(\/|-)(0[1-9]|1[0-2])\2([0-2][0-9]|3[0-1])$'
        tabla = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"])
        print(tabla, campo)
        print(tabla+"."+campo)
        dfqvd[tabla+"."+campo] = dfqvd[campo]
        #dfqvd = dfqvd.replace({campo: Formato},{campo:"Valido"}, regex=True)
        dfqvd = dfqvd.replace({campo: Formato},{campo:"Valido"}, regex=True)
        tabla_campo = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"]+"."+df_param_.iloc[i]["Campo"])
        #Lleamos los valores nulos con "No valido"
        dfqvd1 = dfqvd.fillna("Nulo")
        listas1.append([tabla_campo,campo])
      elif df_param_.iloc[i]["Tipo_dato"] == 'aa/mm/dd':
        campo = (df_param_.iloc[i]["Campo"])
        Formato = '^(\d{2})(\/|-)(0[1-9]|1[0-2])\2([0-2][0-9]|3[0-1])$'
        tabla = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"])
        print(tabla, campo)
        print(tabla+"."+campo)
        dfqvd[tabla+"."+campo] = dfqvd[campo]
        #dfqvd = dfqvd.replace({campo: Formato},{campo:"Valido"}, regex=True)
        dfqvd = dfqvd.replace({campo: Formato},{campo:"Valido"}, regex=True)
        tabla_campo = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"]+"."+df_param_.iloc[i]["Campo"])
        #Lleamos los valores nulos con "No valido"
        dfqvd1 = dfqvd.fillna("Nulo")
        listas1.append([tabla_campo,campo])
      elif df_param_.iloc[i]["Tipo_dato"] == 'aa-mm-dd':
        campo = (df_param_.iloc[i]["Campo"])
        Formato = '^(\d{2})(\/|-)(0[1-9]|1[0-2])\2([0-2][0-9]|3[0-1])$'
        tabla = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"])
        print(tabla, campo)
        print(tabla+"."+campo)
        dfqvd[tabla+"."+campo] = dfqvd[campo]
        #dfqvd = dfqvd.replace({campo: Formato},{campo:"Valido"}, regex=True)
        dfqvd = dfqvd.replace({campo: Formato},{campo:"Valido"}, regex=True)
        tabla_campo = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"]+"."+df_param_.iloc[i]["Campo"])
        #Lleamos los valores nulos con "No valido"
        dfqvd1 = dfqvd.fillna("Nulo")
        listas1.append([tabla_campo,campo])
    elif df_param_.iloc[i]["Valores"] == "Regex":  
        campo = (df_param_.iloc[i]["Campo"])
        Formato = (df_param_.iloc[i]["Valores"])
        tabla = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"])
        print(tabla, campo)
        print(tabla+"."+campo)
        dfqvd[tabla+"."+campo] = dfqvd[campo]
        dfqvd = dfqvd.replace({campo: Formato},{campo:"Valido"}, regex=True)
        tabla_campo = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"]+"."+df_param_.iloc[i]["Campo"])
        #Lleamos los valores nulos con "No valido"
        dfqvd1 = dfqvd.fillna("Nulo")
        listas1.append([tabla_campo,campo])      
        
        
  #print(campo)




dbo_Prueba Cargo
dbo_Prueba.Cargo
dbo_Prueba Email
dbo_Prueba.Email
dbo_Prueba celular
dbo_Prueba.celular
dbo_Prueba fecha_Cumpleaños
dbo_Prueba.fecha_Cumpleaños


----------Captura de tabla_campo y campo ----------

In [56]:
a = '([a-zA-Z0-9_\.]+)@([\da-zA-Z\.-]+)\.([a-zA-Z\.]+)\.*([a-zA-Z]*)'
if re.fullmatch(a, '/*%!#"@vooxell.com' ):
  print('valido')
else:
  print('no valido')

no valido


In [265]:
campo_var = pd.DataFrame(listas1,columns=["ValoresCampo","Cruza"])
campo_var

,ValoresCampo,Cruza
0,dbo_Prueba2.celular,celular


----------Creacion un DataFrame que servirá como base para el .CSV final ----------

In [266]:
dfGroup=pd.DataFrame({"Campo":[],"ValoresCampo":[],"Cruza":[],"ValoresValidacion":[]})
dfGroup

,Campo,ValoresCampo,Cruza,ValoresValidacion


----------Funcion creada para dar como no validas los registros que no cumplen con la extresion regular----------

In [267]:
def asd (a):
    if a != "Valido":
        return("No valido")
    else:
        return(a)

----------Ciclo for en el que se agrupan por Campo, ValoresCampo Cruza y ValoresValidacion y se concatenan los resultados de cada formato requerido----------

In [268]:
for i in range(len(df_param_)):
    dfGroup11 = dfqvd1.groupby(["Campo",campo_var.iloc[i]["ValoresCampo"] ,campo_var.iloc[i]["Cruza"] ],as_index= False).agg({"ValoresValidacion":"count"})
    dfGroup11.rename(columns={"Campo":"Campo",campo_var.iloc[i]["ValoresCampo"]:"ValoresCampo",campo_var.iloc[i]["Cruza"]:"Cruza","ValoresValidacion":"ValoresValidacion"},inplace= True )
    dfGroup11["Campo"] = campo_var.iloc[i]["ValoresCampo"]   
    dfGroup = pd.concat([dfGroup,dfGroup11],axis=0)
    dfGroup["ValoresCampo"] = dfGroup["ValoresCampo"].replace("Nulo","")
    dfGroup["Cruza"] = dfGroup["Cruza"].apply(lambda a: asd(a))
    #dfGroup["ValoresValidacion"] = int(dfGroup["ValoresValidacion"])
dfGroup

,Campo,ValoresCampo,Cruza,ValoresValidacion
0,dbo_Prueba2.celular,12345.0,No valido,1.0
1,dbo_Prueba2.celular,12346.0,No valido,1.0
2,dbo_Prueba2.celular,12347.0,No valido,1.0
3,dbo_Prueba2.celular,12348.0,No valido,1.0
4,dbo_Prueba2.celular,12349.0,No valido,1.0
5,dbo_Prueba2.celular,12350.0,No valido,1.0


In [72]:
dfGroup.to_excel('example1.xlsx', sheet_name='example')

----------Exportar a csv----------

In [ ]:
pathCSV = (dfRutaCSV.iloc[0]["Valor"]) 
pathCSV = pathCSV + "\\"  +tabla + ".csv"
#print(pathCSV)

NameError: ignored

In [ ]:
dfGroup.to_csv(pathCSV, index=False)

----------Mensaje final----------

In [ ]:
print("Archivo exportado, revisar ruta  " + pathCSV)

Archivo exportado, revisar ruta  C:\QVD\Python\dbo_OrderDetail.csv
